In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pm4py==2.2.4
!pip install --upgrade tensorflow==2.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 748.9/748.9 KB 55.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.0 MB/s eta 0:00:00
  Created wheel for pyvis: filename=pyvis-0.3.1-py3-none-any.whl size=755850 sha256=d2deba66e6ff3b9c4bddf976e5bb4b90b0299155f1637e5e5aea07509ad0edbf
  Stored in directory: /root/.cache/pip/wheels/a4/0c/61/8469ca276f96ab772c3acc7f47d71e9737cbdf6f446f017f48
  Created wheel for stringdist: filename=StringDist-1.0.9-cp38-cp38-linux_x86_64.whl size=24941 sha256=08748ae4213dbc183b3894666683ca1da97c3dca0c3fc1d8ce46c674752f

In [9]:
import os

In [10]:
#Changing the directory
os.chdir('/content/drive/MyDrive/Process mining course/Part 2 Project-2')

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import sklearn

from pydream.predictive.nap.NAP import NAP, multiclass_roc_auc_score
import pickle
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support,precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import MinMaxScaler

from pydream.EnhancedPN import EnhancedPN
from pm4py.objects.petri.importer import importer as pnml_importer
from backend.stat.ci_auc import calculate_auc_ci

In [12]:
train_json = "tss_heart_log_train0.json"
valid_json ="tss_heart_log_val0.json"
test_json = "tss_heart_log_test0.json"

In [13]:
train_algo = NAP(tss_train_file=train_json, tss_test_file=valid_json, options={"n_epochs" : 2})
train_algo.train(checkpoint_path="chk_points", name="HF_model", save_results=True)

Xtrain shape: (8403, 1920)
X2train shape: (8403, 3)
Severity train shape: (8403, 58)
X2test shape: (1531, 3)
Xtest shape: (1531, 1920)
Severity test shape: (1531, 58)
Ytrain shape: (8403, 2)
Ytest shape: (1531, 2)
Class weights :  <module 'sklearn.utils.class_weight' from '/usr/local/lib/python3.8/dist-packages/sklearn/utils/class_weight.py'>
Train on 8403 samples, validate on 1531 samples
Epoch 1/2
8403/8403 [==============================] - ETA: 0s - loss: 0.8334 - acc: 0.1632test_prec_mean 0.08001306335728282
test_rec_mean 0.5
test_fscore_mean 0.13795045045045046
test_auc_mean 0.5

Epoch 00001: test_rec_mean improved from -inf to 0.50000, saving model to chk_points/HF_model_split_weights.hdf5
8403/8403 [==============================] - 17s 2ms/sample - loss: 0.8334 - acc: 0.1632 - val_loss: 1.4981 - val_acc: 0.1600 - test_acc: 0.1600 - test_prec_weighted: 0.0256 - test_rec_weighted: 0.1600 - test_loss: 1.4981 - test_fscore_weighted: 0.0442 - test_auc_weighted: 0.5000 - test_prec_m

In [14]:
algo = NAP()
algo.loadModel(path="chk_points", name="HF_model")

X_test, X2_test, severity_test, Y_test = algo.loadData(test_json)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [15]:
stdScaler = MinMaxScaler()
stdScaler.fit(X_test)
X_test = stdScaler.transform(X_test)
stdScaler.fit(X2_test)
X2_test = stdScaler.transform(X2_test)
stdScaler.fit(severity_test)
severity_test = stdScaler.transform(severity_test)

Y_test_int = np.array([int(i) for i in Y_test])
y_prob = algo.model.predict([X_test,X2_test,severity_test])
y_pred = np.argmax(y_prob, axis=1)

test_acc = accuracy_score(Y_test_int, y_pred, normalize=True)

In [16]:
y_true =Y_test_int
y_probscore = []
list_of_lists = []
for x in y_prob:
    list_of_lists.append(list(x))
    y_probscore.append(list(x)[1])


y_score = np.array(y_probscore)
y_pred = np.around(y_pred)
y_true = np.array(y_true)
auc, auc_cov, ci = calculate_auc_ci(y_true,y_score,y_pred, alpha=0.95)
print("AUC Score: ",auc)
print("AUC_cov: ", auc_cov)
print("confidence interval: ",ci)

*** Confusion Matrix ***
true\pred     0     1 
        0   0.0 237.0 
        1   0.0  57.0 

AUC Score:  0.7058997705233547
AUC_cov:  0.0012385729845834541
confidence interval:  [0.63692204 0.7748775 ]
